### Import Libraries

In [1]:
import os
import boto3

import pandas as pd
import numpy as np

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark
                                                                        
role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()


classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = SparkSession.builder.config("spark.driver.extraClassPath", classpath)\
    .master("local[*]").getOrCreate()
    
spark

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as fn
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.mllib.util import MLUtils

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml import Pipeline
from pyspark.ml import Pipeline
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.ml.regression import RandomForestRegressor

### Read file

In [3]:
cdc = spark.read.csv('output.csv', header = True)

In [6]:
cdc.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_1989_revision: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- education_reporting_flag: string (nullable = true)
 |-- month_of_death: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age_recode_52: string (nullable = true)
 |-- place_of_death_and_decedents_status: string (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- day_of_week_of_death: string (nullable = true)
 |-- current_data_year: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- manner_of_death: string (nullable = true)
 |-- method_of_disposition: string (nullable = true)
 |-- autopsy: string (nullable = true)
 |-- activity_code: string (nullable = true)
 |-- place_of_injury_for_causes_w00_y34_except_y06_and_y07_: string (nullable = true)
 |-- 358_cause_recode: string (nullable = true)
 |-- 113_cause_recode: string (nullable = true)
 |-- 130_infant_cause_recode: 

### Impute null values

In [7]:
cdc = cdc.fillna({'place_of_injury_for_causes_w00_y34_except_y06_and_y07_' : '10'})
cdc = cdc.fillna({'130_infant_cause_recode' : '000'})
cdc = cdc.fillna({'activity_code' : '10'})
cdc = cdc.fillna({'manner_of_death' : '8'})
cdc = cdc.fillna({'Place_of_death_and_decedents_status': '7'})

In [8]:
cdc = cdc.fillna({'record_condition_1': 'Other'})
cdc = cdc.fillna({'record_condition_2': 'Other'})
cdc = cdc.fillna({'record_condition_3': 'Other'})
cdc = cdc.fillna({'record_condition_4': 'Other'})

cdc = cdc.fillna({'entity_condition_1': 'Other'})
cdc = cdc.fillna({'entity_condition_2': 'Other'})
cdc = cdc.fillna({'entity_condition_3': 'Other'})
cdc = cdc.fillna({'entity_condition_4': 'Other'})

### Combine other values of methods_of_disposition

In [9]:
cdc = cdc.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'R' , 'O'))
cdc = cdc.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'E' , 'O'))
cdc = cdc.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'D' , 'O'))
cdc = cdc.withColumn('method_of_disposition', regexp_replace('method_of_disposition', 'U' , 'O'))
cdc = cdc.withColumn('Place_of_death_and_decedents_status', regexp_replace('Place_of_death_and_decedents_status', '9' , '7'))

In [14]:
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '00' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '01' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '02' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '03' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '04' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '05' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '06' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '07' , '1'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '08' , '1'))

cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '09' , '2'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '10' , '2'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '11' , '2'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '12' , '3'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '13' , '4'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '14' , '4'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '15' , '5'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '16' , '6'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '17' , '7'))
cdc = cdc.withColumn('education_1989_revision', regexp_replace('education_1989_revision', '99' , '9'))



### Combine the two education revisions

In [15]:
cdc = cdc.withColumn("education_2003_revision", coalesce(cdc.education_2003_revision,cdc.education_1989_revision))

In [16]:
# To remove rows coerced while combining yearly datasets
cdc = cdc.filter(cdc.education_2003_revision.isNotNull())

In [21]:
cdc = cdc.drop('detail_age_type','detail_age', 'age_substitution_flag','age_recode_27', 'age_recode_12',
               'infant_age_recode_22', 'icd_code_10th_revision')
cdc = cdc.drop('education_1989_revision', 'education_reporting_flag','bridged_race_flag', 'race_imputation_flag', 
               'hispanic_origin')
cdc = cdc.drop('record_condition_20' , 'entity_condition_20', 'entity_condition_19', 'entity_condition_18',
               'entity_condition_17', 'record_condition_19', 'record_condition_18', 'record_condition_16',
               'record_condition_17','record_condition_15','record_condition_14','record_condition_13',
               'record_condition_12','record_condition_11','record_condition_10','record_condition_9',
               'record_condition_8','record_condition_7' , 'record_condition_6', 'record_condition_5',
               'entity_condition_16', 'entity_condition_15','entity_condition_14', 'entity_condition_13',
               'entity_condition_12', 'entity_condition_11', 'entity_condition_10', 'entity_condition_9',
               'entity_condition_8', 'entity_condition_7', 'entity_condition_6', 'entity_condition_5')

## Selecting data for firearms

In [28]:
new_cdc = cdc.filter((cdc['358_cause_recode'] == '429') | (cdc['358_cause_recode'] == '435') | (cdc['358_cause_recode'] == '446')
                       | (cdc['358_cause_recode'] == '450') | (cdc['358_cause_recode'] == '451') | (cdc['358_cause_recode'] == '407'))
new_cdc = new_cdc.drop('number_of_entity_axis_conditions','number_of_record_axis_conditions', '39_cause_recode',
                      '130_infant_cause_recode','113_cause_recode',  'month_of_death','current_data_year','day_of_week_of_death')

In [31]:
from pyspark.sql.functions import lit
new_cdc = new_cdc.withColumn("suicide", lit(0))

+---------------+-----------------------+---+-------------+-----------------------------------+--------------+--------------+---------------+---------------------+-------+-------------+------------------------------------------------------+----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----+-------------+-------------+--------------------------+-------+
|resident_status|education_2003_revision|sex|age_recode_52|Place_of_death_and_decedents_status|marital_status|injury_at_work|manner_of_death|method_of_disposition|autopsy|activity_code|place_of_injury_for_causes_w00_y34_except_y06_and_y07_|358_cause_recode|entity_condition_1|entity_condition_2|entity_condition_3|entity_condition_4|record_condition_1|record_condition_2|record_condition_3|record_condition_4|race|race_recode_3|race_recode_5|hispanic_originrace_recode|suicide|
+---------------+-----------------------+---

In [33]:
new_cdc = new_cdc.withColumn('suicide', 
                     when(new_cdc['manner_of_death']== '2' , 1).otherwise(new_cdc['suicide']))

In [51]:
new_cdc = new_cdc.drop('manner_of_death','entity_condition_1','entity_condition_2','entity_condition_3','entity_condition_4',
                       'record_condition_1','record_condition_2','record_condition_3','record_condition_4','358_cause_recode')

In [71]:
new_cdc = new_cdc.withColumn("suicide", new_cdc["suicide"].cast(StringType()))

# Use stringindexer

In [76]:
indexer = StringIndexer(inputCol="resident_status", outputCol="resident_statusIndex").fit(new_dc)
new_df = indexer.transform(new_dc)
indexer = StringIndexer(inputCol="education_2003_revision", outputCol="education_2003_revisionIndex").fit(new_df)
new_df = indexer.transform(new_df)
indexer = StringIndexer(inputCol="sex", outputCol="sexIndex").fit(new_df)
new_df =indexer.transform(new_df)
indexer = StringIndexer(inputCol="age_recode_52", outputCol="age_recode_52Index").fit(new_df)
new_df =indexer.transform(new_df)
indexer = StringIndexer(inputCol="Place_of_death_and_decedents_status", outputCol="Place_of_death_and_decedents_statusIndex").fit(new_cdc)
new_df = indexer.transform(new_cdc)
indexer = StringIndexer(inputCol="marital_status", outputCol="marital_statusIndex").fit(new_df)
new_df = indexer.transform(new_df)
indexer = StringIndexer(inputCol="injury_at_work", outputCol="injury_at_workIndex").fit(new_df)
new_df =indexer.transform(new_df)
indexer = StringIndexer(inputCol="autopsy", outputCol="autopsyIndex").fit(new_df)
new_df =indexer.transform(new_df)
indexer = StringIndexer(inputCol="method_of_disposition", outputCol="method_of_dispositionIndex").fit(new_df)
new_df =indexer.transform(new_df)
indexer = StringIndexer(inputCol="activity_code", outputCol="activity_codeIndex").fit(new_df)
new_df = indexer.transform(new_df)
indexer = StringIndexer(inputCol="place_of_injury_for_causes_w00_y34_except_y06_and_y07_", outputCol="place_of_injury_for_causes_w00_y34_except_y06_and_y07_Index").fit(new_df)
new_df = indexer.transform(new_df)
indexer = StringIndexer(inputCol="race", outputCol="raceIndex").fit(new_df)
new_df =indexer.transform(new_df)
indexer = StringIndexer(inputCol="race_recode_3", outputCol="race_recode_3Index").fit(new_df)
new_df =indexer.transform(new_df)

'indexer = StringIndexer(inputCol="Place_of_death_and_decedents_status", outputCol="Place_of_death_and_decedents_statusIndex").fit(new_cdc)\nnew_df = indexer.transform(new_cdc)\nindexer = StringIndexer(inputCol="marital_status", outputCol="marital_statusIndex").fit(new_df)\nnew_df = indexer.transform(new_df)\nindexer = StringIndexer(inputCol="injury_at_work", outputCol="injury_at_workIndex").fit(new_df)\nnew_df =indexer.transform(new_df)\nindexer = StringIndexer(inputCol="autopsy", outputCol="autopsyIndex").fit(new_df)\nnew_df =indexer.transform(new_df)\nindexer = StringIndexer(inputCol="method_of_disposition", outputCol="method_of_dispositionIndex").fit(new_df)\nnew_df =indexer.transform(new_df)\nindexer = StringIndexer(inputCol="activity_code", outputCol="activity_codeIndex").fit(new_df)\nnew_df = indexer.transform(new_df)\nindexer = StringIndexer(inputCol="place_of_injury_for_causes_w00_y34_except_y06_and_y07_", outputCol="place_of_injury_for_causes_w00_y34_except_y06_and_y07_Index"

In [78]:
indexer = OneHotEncoder(inputCol="resident_statusIndex", outputCol="resident_statusVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="education_2003_revisionIndex", outputCol="education_2003_revisionVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="sexIndex", outputCol="sexVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="age_recode_52Index", outputCol="age_recode_52Vec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="Place_of_death_and_decedents_statusIndex", outputCol="Place_of_death_and_decedents_statusVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="marital_statusIndex", outputCol="marital_statusVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="injury_at_workIndex", outputCol="injury_at_workVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="autopsyIndex", outputCol="autopsyVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="method_of_dispositionIndex", outputCol="method_of_dispositionVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="activity_codeIndex", outputCol="activity_codeVec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="place_of_injury_for_causes_w00_y34_except_y06_and_y07_Index", outputCol="place_of_injury_for_causes_w00_y34_except_y06_and_y07_Vec")
new_df = indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="raceIndex", outputCol="raceVec")
new_df =indexer.transform(new_df)
indexer = OneHotEncoder(inputCol="race_recode_3Index", outputCol="race_recode_3Vec")
new_df =indexer.transform(new_df)

In [105]:
new_df.columns

['resident_status',
 'education_2003_revision',
 'sex',
 'age_recode_52',
 'Place_of_death_and_decedents_status',
 'marital_status',
 'injury_at_work',
 'method_of_disposition',
 'autopsy',
 'activity_code',
 'place_of_injury_for_causes_w00_y34_except_y06_and_y07_',
 'race',
 'race_recode_3',
 'race_recode_5',
 'hispanic_originrace_recode',
 'suicide',
 'Place_of_death_and_decedents_statusIndex',
 'marital_statusIndex',
 'injury_at_workIndex',
 'autopsyIndex',
 'method_of_dispositionIndex',
 'activity_codeIndex',
 'place_of_injury_for_causes_w00_y34_except_y06_and_y07_Index',
 'raceIndex',
 'race_recode_3Index',
 'resident_statusIndex',
 'education_2003_revisionIndex',
 'sexIndex',
 'age_recode_52Index',
 'resident_statusVec',
 'education_2003_revisionVec',
 'sexVec',
 'age_recode_52Vec',
 'Place_of_death_and_decedents_statusVec',
 'marital_statusVec',
 'injury_at_workVec',
 'autopsyVec',
 'method_of_dispositionVec',
 'activity_codeVec',
 'place_of_injury_for_causes_w00_y34_except_y06_

In [118]:
new_df.printSchema()

root
 |-- resident_status: string (nullable = true)
 |-- education_2003_revision: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- age_recode_52: string (nullable = true)
 |-- Place_of_death_and_decedents_status: string (nullable = false)
 |-- marital_status: string (nullable = true)
 |-- injury_at_work: string (nullable = true)
 |-- method_of_disposition: string (nullable = true)
 |-- autopsy: string (nullable = true)
 |-- activity_code: string (nullable = false)
 |-- place_of_injury_for_causes_w00_y34_except_y06_and_y07_: string (nullable = false)
 |-- race: string (nullable = true)
 |-- race_recode_3: string (nullable = true)
 |-- race_recode_5: string (nullable = true)
 |-- hispanic_originrace_recode: string (nullable = true)
 |-- suicide: integer (nullable = false)
 |-- Place_of_death_and_decedents_statusIndex: double (nullable = true)
 |-- marital_statusIndex: double (nullable = true)
 |-- injury_at_workIndex: double (nullable = true)
 |-- autopsyIndex: double (nu

In [80]:
assemblerInputs = ['resident_statusVec',
 'education_2003_revisionVec',
 'sexVec',
 'age_recode_52Vec',
 'Place_of_death_and_decedents_statusVec',
 'marital_statusVec',
 'injury_at_workVec',
 'autopsyVec',
 'method_of_dispositionVec',
 'activity_codeVec',
 'place_of_injury_for_causes_w00_y34_except_y06_and_y07_Vec',
 'raceVec',
 'race_recode_3Vec']

In [81]:
assemblerInputs

['resident_statusVec',
 'education_2003_revisionVec',
 'sexVec',
 'age_recode_52Vec',
 'Place_of_death_and_decedents_statusVec',
 'marital_statusVec',
 'injury_at_workVec',
 'autopsyVec',
 'method_of_dispositionVec',
 'activity_codeVec',
 'place_of_injury_for_causes_w00_y34_except_y06_and_y07_Vec',
 'raceVec',
 'race_recode_3Vec']

In [84]:
label_string_index = StringIndexer(inputCol="suicide", outputCol="label").fit(new_df)
new_df = label_string_index.transform(new_df)

In [85]:
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
new_df = assembler.transform(new_df)  

In [86]:
training, test = new_df.randomSplit([0.7, 0.3], seed = 11)

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=10)

# Train model.  This also runs the indexers.
model = rf.fit(training)

# Make predictions.
predictions = model.transform(test)

In [120]:
meta = [f.metadata 
    for f in new_df.schema.fields 
    if f.name == 'features'][0]
features_name_ind = meta['ml_attr']['attrs']['binary']

[{'idx': 0, 'name': 'resident_statusVec_1'},
 {'idx': 1, 'name': 'resident_statusVec_2'},
 {'idx': 2, 'name': 'resident_statusVec_3'},
 {'idx': 3, 'name': 'education_2003_revisionVec_3'},
 {'idx': 4, 'name': 'education_2003_revisionVec_2'},
 {'idx': 5, 'name': 'education_2003_revisionVec_4'},
 {'idx': 6, 'name': 'education_2003_revisionVec_6'},
 {'idx': 7, 'name': 'education_2003_revisionVec_1'},
 {'idx': 8, 'name': 'education_2003_revisionVec_5'},
 {'idx': 9, 'name': 'education_2003_revisionVec_9'},
 {'idx': 10, 'name': 'education_2003_revisionVec_7'},
 {'idx': 11, 'name': 'sexVec_M'},
 {'idx': 12, 'name': 'age_recode_52Vec_30'},
 {'idx': 13, 'name': 'age_recode_52Vec_31'},
 {'idx': 14, 'name': 'age_recode_52Vec_32'},
 {'idx': 15, 'name': 'age_recode_52Vec_36'},
 {'idx': 16, 'name': 'age_recode_52Vec_35'},
 {'idx': 17, 'name': 'age_recode_52Vec_33'},
 {'idx': 18, 'name': 'age_recode_52Vec_34'},
 {'idx': 19, 'name': 'age_recode_52Vec_29'},
 {'idx': 20, 'name': 'age_recode_52Vec_37'},
 

In [133]:
feature_names = []
for i in model.featureImportances.indices:
    feature_names.append(features_name_ind[i]['name'])

In [146]:
df_values = [('feature_names', feature_names), ('variable_imp', model.featureImportances.values)]
inference_df = pd.DataFrame.from_items(df_values)
inference_df.sort_values(by = 'variable_imp', ascending= False)

,feature_names,variable_imp
54,raceVec_02,0.209822
53,raceVec_01,0.152422
61,race_recode_3Vec_3,0.100893
45,place_of_injury_for_causes_w00_y34_except_y06_...,0.096556
60,race_recode_3Vec_1,0.073787
39,autopsyVec_N,0.072708
47,place_of_injury_for_causes_w00_y34_except_y06_...,0.065462
30,Place_of_death_and_decedents_statusVec_2,0.041618
38,autopsyVec_Y,0.038952
28,Place_of_death_and_decedents_statusVec_4,0.032348


In [130]:
model.featureImportances[4]

0.0038043378849072227

In [ ]:
meta = [f.metadata 
    for f in preppedDataDF.schema.fields 
    if f.name == 'features'][0]
features_name_ind = meta['ml_attr']['attrs']['binary']
features_name_ind

In [111]:
tp = selected.where(selected["label"] == 1).where(selected["prediction"] == 1).count()
tn = selected.where(selected["label"] == 0).where(selected["prediction"] == 0).count()
fp = selected.where(selected["label"] == 0).where(selected["prediction"] == 1).count()
fn = selected.where(selected["label"] == 1).where(selected["prediction"] == 0).count()

In [112]:
accuracy = (tp + tn) * 100 / (tp + tn + fp + fn)
accuracy

82.18496735053026

In [113]:
precision = tp / (tp + fp)
precision

0.8780351313539562

In [114]:
recall = tp / (tp + fn)
recall

0.6484513121943379

In [115]:
f1_score = 2 * precision * recall / (precision + recall)
f1_score

0.7459784991679654